### RQ2: Term frequency extraction and analysis to discover the most common words used in the commit messages

In [20]:
import pandas as pd
import chardet
import nltk
from nltk.corpus import stopwords
import pickle
import pyarrow.parquet as pq
import numpy as np
import os
import vaex as vx
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st
import math
import glob
from scipy.optimize import curve_fit
import time
from random import sample
from platform import python_version

In [21]:
# VAEX DATAFRAME
df= []

#### Using VAEX dataframes for larger 

In [22]:
# READ ALL THE PARQUET FILE
df = vx.open('/data/fdac21/eawoc//processed_data/*.parquet')

In [23]:
# COLUMN NAMES
columns = ["projectID", "commitID", "author", "committer", 
           "authorTime", "committerTime", "authorTimezone", 
           "committerTimezone", "commitMessage"]

In [24]:
# Review the data
df.head()

#,projectID,commitID,author,committer,authorTime,committerTime,authorTimezone,committerTimezone,commitMessage
0,wildfly_wildfly,00001a1a9d1552584eee5e62ab11d7b86408c0d8,ochaloup <ochaloup@redhat.com>,ochaloup <ochaloup@redhat.com>,1488898509,1489149067,+0100,+0100,'[JBTM-2858] adding JAXRS filter to import trans...
1,wildfly_wildfly,00010938ae7227834e2c69a9c3c71c95d3f3483d,Stuart Douglas <stuart.w.douglas@gmail.com>,Stuart Douglas <stuart.w.douglas@gmail.com>,1458105079,1458105079,+1100,+1100,WFCORE-1436 Undertow 1.3.19.Final
2,wildfly_wildfly,000183c08f9e7d9770ad8216acb565552da7bfb0,Jeff Mesnil <jmesnil@gmail.com>,Jeff Mesnil <jmesnil@gmail.com>,1332344580,1332436142,+0100,+0100,'[JBPAPP-8184] Connection factory failure\\n* add...
3,wildfly_wildfly,0001a3f7a195b75e563280609636faa249877036,jaikiran <jaikiran.pai@gmail.com>,baranowb <baranowb@gmail.com>,1328294482,1328611298,+0530,+0100,Upgrade to 2.0.0 of jboss-ejb3-ext-api
4,wildfly_wildfly,00024ccfa6fa6fd6426de7bb1d8d494798b384fe,James R. Perkins <jperkins@redhat.com>,GitHub <noreply@github.com>,1583899415,1583899415,-0700,-0700,'Merge pull request #13102 from kabir/WFLY-13222...
5,wildfly_wildfly,0002dbdc7b6cb98e6387cc5f0d11bb93229c5381,Kabir Khan <kkhan@redhat.com>,GitHub <noreply@github.com>,1491037874,1491037874,+0100,+0100,'Merge pull request #9883 from jbosstm/WFLY-8485...
6,wildfly_wildfly,000325d9bb03ceffbe9e7b4756a5566b273040d1,Brian Stansberry <brian.stansberry@redhat.com>,Brian Stansberry <brian.stansberry@redhat.com>,1503172204,1505862581,-0500,-0500,Minor cleanup
7,wildfly_wildfly,0003c4792f52061661cc9db421c7741bc4df8284,Richard Achmatowicz <rachmatowicz@jboss.com>,Paul Ferraro <paul.ferraro@redhat.com>,1370975878,1374591017,-0400,-0400,"[WFLY-1430] Add in XML processing, test cases."
8,wildfly_wildfly,0004ba7ca70b15a48fe6460ec5375c86cd8dc19d,Scott Marlow <smarlow@redhat.com>,Scott Marlow <smarlow@redhat.com>,1463689533,1473273893,-0400,-0400,'add org.wildfly.extension.nosql.common dependen...
9,wildfly_wildfly,000703aae4a2cc2cc0f1c4a9a03ae391bb76def0,Pelisse Romain <belaran@redhat.com>,GitHub <noreply@github.com>,1470623929,1470623929,+0000,+0000,'Merge 03a1e02b4aa392ec3d8914846eee615232f45d89 ...


In [25]:
# list of project names 
project_names = df.projectID.unique()
print(project_names)

['saqib-ali_hivemined', 'git.kernel.org_pub_scm/public-inbox/vger.kernel.org/linux-media/0', 'RIOT-OS_RIOT', 'trattkantarell_src', 'sahil28297_erpnext', 'ThePiGuy0_android_device_motorola_msm8226-common', 'git.kernel.org_pub_scm/public-inbox/vger.kernel.org/lkml/0', 'jdzs1_b', 'ZIDAN44_linux-stable', 'timhuynh94_231AS', 'Leaflet_Leaflet', 'gitorious.org_netbeans/main-silver', 'discourse_discourse', 'nanoframework_ChibiOS-bkp', 'AnnaLut_abs3', 'civicrm_civicrm-core', 'thomasNDS_lbl', 'gentoo_gentoo', 'neo4j_neo4j', 'maritimerabroad_gitfiti', 'dotnet_aspnetcore', 'geos4s_18S856162', 'learn-co-curriculum_python-for-loops-lab', 'jthub_ega-file-transfer-to-collab-1-jtracker', 'denis-gz_qtbase', 'Alsan_turing-chunk07', 'CRAZYFAKE_handiwork', 'i17c_ecode', 'Reese-D_my_emacs', 'gitlab.com_Ponce_slackbuilds', 'jthub_ega-file-transfer-to-collab-2-jtracker', 'stoupoun_p2-breathe', 'Waffiliate_repoForSynch', 'jthub_ega-file-transfer-to-collab-5-jtracker', 'Arshiamidos_84ck1N617', 'mono_mono', 'app

### [2] Term frequency extraction and analysis to discover the most common words used in the commit messages

In [102]:
# Use the df data frame for the analysis from above. The commitMessage column contains all the 
# text of the projects. Recommended to work on a smaller sample of commitMessage first. 

In [83]:
# sample 100 projects
project_samples = sample(project_names, 100)

In [84]:
allc=[]
for project_name in project_samples:
    try:
        print(project_name)
        project = df[df['projectID'] == project_name]
        length = len(project)    
        if(length > 100):
            print('---')
            pp = project.sample(n= 100)
            commit_messages = pp[['commitMessage']].copy()
            cm = commit_messages['commitMessage'].tolist()
            allc.extend(cm)
            print(cm)
            print('--------------------')
    except:
        print('continue')
        
import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(allc, fp)


jakirkham_feedstocks
---
['Updated the xorg-x11-proto-devel-cos7-aarch64 feedstock.', 'Updated the feedstock listing.', 'Updated the pyiron_base feedstock.', 'Updated the sphinx-autoapi feedstock.', 'Updated the humanize feedstock.', 'Updated the language-tags feedstock.', 'Updated the qdatamatrix feedstock.', 'Updated the tomviz feedstock.', 'Updated the rasterio feedstock.', 'Updated the feedstock listing.', 'Updated the nvcc feedstock.', 'Updated the gobject-introspection feedstock.', 'Updated the psutil feedstock.', 'Updated the r-sjstats feedstock.', 'Updated the r-randomizr feedstock.', 'Updated the chealpix feedstock.', 'Updated the r-epi feedstock.', 'Updated the pdpipe feedstock.', 'Updated the apispec feedstock.', 'Updated the openpmd-viewer feedstock.', 'Updated the r-outliers feedstock.', 'Updated the vim feedstock.', 'Updated the charls feedstock.', 'Updated the pyemma_tutorials feedstock.', 'Updated the r-rebus.datetimes feedstock.', 'Updated the chardet feedstock.', 'Upd

---
['A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A commit a day keeps the girlfriend away.', 'A co

### [5.2] Develop static and interactive visualizations to demonstrate the findings.

In [93]:
# The pickle file read might have some issues like None. which should be cleaned before the tf-idf calculation

import pickle
mynewlist=[]
updatedlist=[]
with open('outfile', 'rb') as f:
    mynewlist = pickle.load(f)
for i in mynewlist:
    if i is not None:
        updatedlist.append(i)

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 

vectorizer = TfidfVectorizer(stop_words=text.ENGLISH_STOP_WORDS)
response = vectorizer.fit_transform(updatedlist)
df_tfidf_sklearn = pd.DataFrame(response.toarray(),columns=vectorizer.get_feature_names())
df_tfidf_sklearn
with open('rq2-tf-idf-results.pickle', 'wb') as fp:
    pickle.dump(df_tfidf_sklearn, fp)

In [95]:
df_tfidf_sklearn.describe()

,00,000,0039d316,0056676,0081def497697c7bb82befcdcd95839d91ca237fa51c3d972fca3f46c9466df3,00c58bce0d2d1e0d13ee29656c8c26d7d9ce70bf,00cc23d1f72b,00ee22c28915d111ba415750a3311d7678fd1206,01,0100,...,ðµð½ð,ðµð½ð³ñ,ðµñ,ðºð¾ð³ð,ð¼ðµð,ð½ðµ,ð½ðµð¼,ð¾ð,ð¾ð½ð,ð¾ñ
count,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,...,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000,9547.000000
mean,0.003066,0.000041,0.000088,0.000028,0.000074,0.000038,0.000043,0.000012,0.001821,0.000035,...,0.000021,0.000074,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000042,0.000042
std,0.041737,0.002996,0.003916,0.002717,0.007237,0.003702,0.004191,0.001213,0.025464,0.002458,...,0.002047,0.007237,0.002047,0.002047,0.002047,0.002047,0.002047,0.002047,0.004094,0.004094
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.755087,0.260888,0.207271,0.265472,0.707107,0.361738,0.409496,0.118475,0.624576,0.186174,...,0.200000,0.707107,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.400000,0.400000
